In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults

tools = [TavilySearchResults(max_results=1)]

In [14]:
from langgraph.prebuilt import ToolExecutor

tool_executor = ToolExecutor(tools)


In [4]:
import vertexai
vertexai.init(project="gemini-api-428204", location="us-central1")

In [5]:
from langchain_google_vertexai import ChatVertexAI

In [24]:
llm = ChatVertexAI(model_name="gemini-pro")

In [8]:
model = llm.bind_tools(tools)

In [9]:
import operator
from typing import Annotated, Sequence, TypedDict

from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]


In [26]:
from langchain_core.messages import ToolMessage

from langgraph.prebuilt import ToolInvocation


def should_continue(state):
    messages = state["messages"]
    last_message = messages[-1]
    if not last_message.tool_calls:
        return "end"
    else:
        return "continue"


def call_model(state):
    messages = state["messages"]
    response = model.invoke(messages)
    return {"messages": [response]}


def call_tool(state):
    messages = state["messages"]
    last_message = messages[-1]
    tool_call = last_message.tool_calls[0]
    action = ToolInvocation(
        tool=tool_call["name"],
        tool_input=tool_call["args"],
    )
    print(action)
    response = tool_executor.invoke(action)
    function_message = ToolMessage(
        content=str(response), name=action.tool, tool_call_id=tool_call["id"]
    )
    return {"messages": [function_message]}


In [27]:
from langgraph.graph import END, StateGraph, START

workflow = StateGraph(AgentState)

workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)

workflow.add_edge(START, "agent")

workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "action",
        "end": END,
    },
)

workflow.add_edge("action", "agent")

app = workflow.compile()


In [31]:
from langchain_core.messages import HumanMessage

inputs = {"messages": [HumanMessage(content="what is the vector database")]}
app.invoke(inputs)


tool='tavily_search_results_json' tool_input={'query': 'what is the vector database'}


{'messages': [HumanMessage(content='what is the vector database'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "what is the vector database"}'}}, response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}], 'usage_metadata': {'prompt_token_count': 59, 'candidates_token_count': 14, 'total_token_count': 73}}, id='run-c9a8be65-3bb1-412e-94f8-c727fc470c9c-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'what is the vector database'}, 'id': '35f4ea73-2510-4d36-9340-987a2f0991fa'}], usage_metadata={'inp

In [32]:
for output in app.stream(inputs):
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")


Output from node 'agent':
---
{'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "what is vector database"}'}}, response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}], 'usage_metadata': {'prompt_token_count': 59, 'candidates_token_count': 13, 'total_token_count': 72}}, id='run-94e25494-7dda-4305-ad0e-d6386ef3f69c-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'what is vector database'}, 'id': 'f89fc7bf-e85d-4aa6-b44b-6464f6f3b233'}], usage_metadata={'input_tokens': 59, 'output_tokens': 